In [1]:
import torch
torch.cuda.empty_cache() 

In [2]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image

In [3]:
import pandas as pd
import pyproj

In [4]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [5]:
from pathlib import Path

In [6]:
import joblib

In [7]:
from mt_function_yl import *

In [8]:
import seaborn as sn

In [9]:
from scipy import stats

In [10]:
from tabulate import tabulate

In [11]:
import statsmodels.api as sm

### Data loading

In [13]:
aa = pd.read_csv('/home/yelhe/script/mt/dataraw/dfaa.csv')

In [200]:
# tt variables

# Distance to features
busl = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_busline.csv')
traml = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_tramrail.csv')
trainl = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_sbahn_h.csv')
plight = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_publiclight.csv')
station = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_station_h.csv')
parkcar = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_parking_car_space.csv')
parktw = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_parking_twowheeler.csv')

pedstrainc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_pedestraincrossing_n.csv') # Greg
stop = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_stopsign_n.csv') # Greg

# Road width & surface information
roadinfo = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_roadsurfacewidth.csv') # Greg
# Speedlimit
speedlimit = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_speedlimit.csv') # Greg

# Traffic volume/exposure
gvm = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_gvm.csv')

# Area statistics zone
trafficarea = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_trafficarea_h.csv')
statzone = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_statzonen_h.csv')

# Curb info
curb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_curb.csv')

# Traffic count
bc1921 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecount1921.csv')
vc1921 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/carcount1921.csv')

In [17]:
ttv = pd.DataFrame()

In [22]:
buslg = busl.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})

In [26]:
buslg = buslg.rename(columns = {'distance': 'dbusl'})
buslg = buslg.drop(['count'], axis = 1)

In [24]:
ttv['Nr'] = aa['Nr']

In [27]:
ttv = ttv.merge(buslg, on = ['Nr'])

In [30]:
tramlg = traml.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
tramlg = tramlg.rename(columns = {'distance': 'dtraml'})
tramlg = tramlg.drop(['count'], axis = 1)

In [32]:
ttv = ttv.merge(tramlg, on = ['Nr'])

In [35]:
trainlg = trainl.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
trainlg = trainlg.rename(columns = {'distance': 'dtrainl'})
trainlg = trainlg.drop(['count'], axis = 1)

In [36]:
ttv = ttv.merge(trainlg, on = ['Nr'])

In [39]:
plightg = plight.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
plightg = plightg.rename(columns = {'distance': 'dplight'})
plightg = plightg.drop(['count'], axis = 1)

In [41]:
ttv = ttv.merge(plightg, on = ['Nr'])

In [44]:
stationg = station.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
stationg = stationg.rename(columns = {'distance': 'dstation'})
stationg = stationg.drop(['count'], axis = 1)

In [46]:
ttv = ttv.merge(stationg, on = ['Nr'])

In [60]:
parkcarg = parkcar.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
parkcarg = parkcarg.rename(columns = {'distance': 'dparkcar'})
parkcarg = parkcarg.drop(['count'], axis = 1)
ttv = ttv.merge(parkcarg, on = ['Nr'])
parktwg = parktw.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
parktwg = parktwg.rename(columns = {'distance': 'dparktw'})
parktwg = parktwg.drop(['count'], axis = 1)
ttv = ttv.merge(parktwg, on = ['Nr'])
pedcrg = pedstrainc.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
pedcrg = pedcrg.rename(columns = {'distance': 'dpedcro'})
pedcrg = pedcrg.drop(['count'], axis = 1)
ttv = ttv.merge(pedcrg, on = ['Nr'])
stopsg = stop.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
stopsg = stopsg.rename(columns = {'distance': 'dstopsign'})
stopsg = stopsg.drop(['count'], axis = 1)
ttv = ttv.merge(stopsg, on = ['Nr'])

In [64]:
curbg = curb.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
curbg = curbg.rename(columns = {'distance': 'dcurb'})
curbg = curbg.drop(['count'], axis = 1)
ttv = ttv.merge(curbg, on = ['Nr'])

In [67]:
curbty = curb[['Nr', 'distance', 'abschluss']]

In [88]:
curbty = curbty.rename(columns = {'abschluss':'curbtype', 'distance':'dcurb'})
curbty = curbty.drop_duplicates()
ttv = ttv.merge(curbty, on = ['Nr', 'dcurb'])

In [73]:
gvmg = gvm.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
gvmg = gvmg.rename(columns = {'distance': 'dgvm'})
gvmg = gvmg.drop(['count'], axis = 1)
ttv = ttv.merge(gvmg, on = ['Nr'])
gvmvalue = gvm[['Nr', 'DWV_ANZ_Q', 'MSP_ANZ_Q', 'ASP_ANZ_Q', 'distance']]
gvmvalue = gvmvalue.rename(columns = {'DWV_ANZ_Q':'gvm_dwv',
                                      'MSP_ANZ_Q':'gvm_msp',
                                      'ASP_ANZ_Q':'gvm_asp',
                                      'distance':'dgvm'})
ttv = ttv.merge(gvmvalue, on = ['Nr', 'dgvm'])

In [90]:
ttv = ttv.drop(['curbtype_x'], axis = 1)
ttv = ttv.drop_duplicates()

In [92]:
ttv = ttv.rename(columns = {'curbtype_y': 'curbtype'})

In [94]:
ttv = ttv.reset_index()

In [97]:
ttv = ttv.drop(['index'], axis = 1)

In [100]:
print(len(statzone['stznr'].unique()),
      len(statzone['qnr'].unique()),
      len(statzone['knr'].unique()))

122 33 12


In [105]:
statzoneg = statzone[['Nr', 'stznr', 'qnr', 'knr']]
statzoneg = statzoneg.rename(columns = {'stznr':'z_stznr', 'qnr':'z_qnr', 'knr':'z_knr'})
ttv = ttv.merge(statzoneg, on = ['Nr'])

In [118]:
trafficarea.columns

Index(['Nr', 'Koord E', 'Koord N', 'lat', 'lon', 'id', 'zonenname', 'zonentyp',
       'zonentyp_t', 'umgesetzt_', 'rechtskrae', 'publiziert', 'objectid',
       'distance'],
      dtype='object')

In [116]:
trafficarea.groupby(['zonentyp', 'zonentyp_t']).size()

zonentyp        zonentyp_t
Begegnungszone  T20            34
Fussgängerzone  T0             32
T30-Zone        T30           284
dtype: int64

In [122]:
tareag = trafficarea[['Nr', 'zonentyp_t', 'distance']]
tareag = tareag.rename(columns = {'zonentyp_t':'trafficarea',
                                  'distance':'dtrafficarea'})
ttv = ttv.merge(tareag, on = ['Nr'])

In [127]:
rig = roadinfo[['Nr', 'road_width', 'surface', 'distance']]
rig = rig.rename(columns = {'road_width':'r_width',
                               'distance':'droad',
                               'surface':'r_surface'})
ttv = ttv.merge(rig, on = ['Nr'])

In [130]:
slg = speedlimit[['Nr', 'speed', 'distance']]
slg = slg.rename(columns = {'speed':'speedlimit',
                            'distance':'droad_sl'})
ttv = ttv.merge(slg, on = ['Nr'])

In [137]:
ttv.columns

Index(['Nr', 'dbusl', 'dtraml', 'dtrainl', 'dplight', 'dstation', 'dparkcar',
       'dparktw', 'dpedcro', 'dstopsign', 'dcurb', 'dgvm', 'gvm_dwv',
       'gvm_msp', 'gvm_asp', 'curbtype', 'z_stznr', 'z_qnr', 'z_knr',
       'trafficarea', 'dtrafficarea', 'r_width', 'r_surface', 'droad',
       'speedlimit', 'droad_sl'],
      dtype='object')

In [151]:
ttv.to_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum.csv', index = False)

In [196]:
ttv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum.csv')

In [202]:
bc1921 = bc1921.drop(['Unnamed: 0'], axis = 1)

In [205]:
list = bc1921.index.values

In [207]:
bc1921 = bc1921.drop(index = [76, 97, 79, 73, 139, 86, 149, 128, 118])

In [209]:
bcg = bc1921.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
bcg = bcg.rename(columns = {'distance': 'd_bicyclecount'})
bcg = bcg.drop(['count'], axis = 1)
bcg = bcg.drop_duplicates()
ttv = ttv.merge(bcg, on = ['Nr'], how = 'left')
bcgv = bc1921[['Nr', 'bicycle_traffic_count', 'distance']]
bcgv = bcgv.rename(columns = {'bicycle_traffic_count':'bicyclecount',
                              'distance':'d_bicyclecount'})
ttv = ttv.merge(bcgv, on = ['Nr', 'd_bicyclecount'], how = 'left')

In [213]:
vc1921 = vc1921.drop(['Unnamed: 0'], axis = 1)

In [214]:
vc1921 = vc1921.drop(index = [42, 40, 76])

In [217]:
vc1921.loc[41, 'car_traffic_count'] = 88424.5

In [220]:
vc1921.loc[39, 'car_traffic_count'] = 87416.5

In [223]:
vc1921.loc[75, 'car_traffic_count'] = 88582.5

In [225]:
vcg = vc1921.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
vcg = vcg.rename(columns = {'distance': 'd_carcount'})
vcg = vcg.drop(['count'], axis = 1)
vcg = vcg.drop_duplicates()
ttv = ttv.merge(vcg, on = ['Nr'], how = 'left')
vcgv = vc1921[['Nr', 'car_traffic_count', 'distance']]
vcgv = vcgv.rename(columns = {'car_traffic_count':'carcount',
                              'distance':'d_carcount'})
ttv = ttv.merge(vcgv, on = ['Nr', 'd_carcount'], how = 'left')

In [226]:
ttv

,Nr,dbusl,dtraml,dtrainl,dplight,dstation,dparkcar,dparktw,dpedcro,dstopsign,...,dtrafficarea,r_width,r_surface,droad,speedlimit,droad_sl,d_bicyclecount,bicyclecount,d_carcount,carcount
0,99900060852597,29.425561,24.211812,106.688897,3.781712,26.868817,35.656105,194.231319,8.128722,712.383770,...,1.794085,4,Hart,1.885184,30.0,1.885149,NaN,NaN,NaN,NaN
1,99900063018611,11.679839,460.233213,1585.606526,2.626894,47.632749,89.812220,329.769619,78.080802,56.381644,...,55.044835,2,Hart,0.700847,0.0,0.700853,NaN,NaN,NaN,NaN
2,99900052723510,80.234787,129.630319,272.559762,9.161328,183.173667,12.535775,80.267681,68.281781,670.054240,...,0.000000,4,Hart,4.735467,30.0,4.735445,NaN,NaN,749.217296,45701.0
3,99900061514043,2.506815,101.196875,409.942474,9.201655,126.478735,3.217583,158.221280,101.609587,1112.567978,...,63.477600,8,Hart,1.688573,60.0,1.688617,NaN,NaN,NaN,NaN
4,99900063037843,45.063084,141.952642,1373.050289,13.201583,161.199806,8.905580,130.926393,125.990176,363.725692,...,0.000000,4,Hart,5.435503,30.0,5.435473,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,99900055905276,11.710725,2.234845,895.476702,0.971010,101.636500,66.327998,371.563385,44.091890,887.327691,...,1.103187,6,Hart,11.255950,50.0,11.255981,2405.574172,6692.00841,580.977396,26432.0
346,99900060670975,172.642072,163.448242,721.773091,6.766016,176.783598,10.735260,212.876883,6.765543,662.060481,...,0.000000,4,Hart,3.385416,30.0,3.385384,NaN,NaN,NaN,NaN
347,99900047367168,272.358546,5.854481,647.324745,7.363513,30.462290,192.688788,156.788057,177.979082,366.608632,...,48.667513,8,Hart,8.463655,0.0,8.463668,NaN,NaN,293.501195,78167.0
348,99900050714619,142.965486,132.833440,438.934821,6.127168,146.742527,6.174056,69.721351,143.169739,631.488493,...,0.000000,3,Hart,3.388330,30.0,3.388310,NaN,NaN,202.338784,26265.0


In [14]:
# ttv.to_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_withtrafficcount.csv', index = False)

In [24]:
ttv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_withtrafficcount.csv')
vfpath = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_velofusspath.csv')

In [25]:
vfpg = vfpath.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
vfpg = vfpg.rename(columns = {'distance': 'dvfpath'})
vfpg = vfpg.drop(['count'], axis = 1)

In [26]:
vfpg

,Nr,dvfpath
0,99900044963840,9.410807
1,99900045510756,0.420799
2,99900045511204,0.437524
3,99900045553013,6.802020
4,99900045612893,7.188158
...,...,...
345,99900063695545,5.926033
346,99900063698497,0.909111
347,99900063992212,4.419567
348,99900064009149,2.945314


In [27]:
ttv = ttv.merge(vfpg, on = ['Nr'])

In [28]:
ttv.to_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_updated0323.csv', index = False)